In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
import matplotlib.pyplot as plt
import gc

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
dataset_train_validate = pd.read_csv("train.csv")
dataset_test = pd.read_csv("test.csv")

# train: 300000 rest for validation
# split after preprocessing
# train = dataset_train_validate[]

In [4]:
dataset_train_validate.shape

(348978, 51)

In [5]:
dataset_train_validate

,transaction_id,num_var_1,num_var_2,num_var_3,num_var_4,num_var_5,num_var_6,num_var_7,cat_var_1,cat_var_2,...,cat_var_34,cat_var_35,cat_var_36,cat_var_37,cat_var_38,cat_var_39,cat_var_40,cat_var_41,cat_var_42,target
0,id_11,2.302632e-08,0.040182,0.0,1.800000e-07,2.302632e-08,2.368421e-08,1.115205e-08,NaN,ce,...,0,0,0,0,0,0,0,0,0,0
1,id_33,7.965789e-06,0.157872,0.0,2.105000e-06,2.769737e-07,7.965789e-06,2.433058e-06,da,tn,...,0,0,0,0,0,0,0,0,0,0
2,id_51,7.828947e-08,0.089140,0.0,3.550000e-07,4.671053e-08,1.052632e-07,4.276014e-07,gf,ce,...,0,0,0,0,0,0,0,0,0,0
3,id_54,7.894737e-08,0.227239,0.0,1.050000e-06,1.381579e-07,2.190789e-07,1.848054e-08,NaN,ce,...,0,0,0,0,0,0,0,0,0,0
4,id_62,3.321053e-06,0.160410,0.0,2.105000e-06,2.769737e-07,3.340789e-06,2.152983e-06,da,tn,...,0,0,0,0,0,0,0,0,0,0
5,id_67,1.953947e-07,0.078566,0.0,3.550000e-07,4.671053e-08,4.407895e-08,9.463310e-08,gf,ce,...,0,0,0,0,0,0,0,0,0,0
6,id_71,1.736842e-07,0.600508,0.0,4.575000e-06,5.986842e-07,1.736842e-07,8.411831e-08,hn,ce,...,0,0,0,0,0,0,0,0,0,0
7,id_88,5.789474e-08,0.152374,0.0,2.105000e-06,2.769737e-07,5.789474e-08,2.453451e-08,da,tn,...,0,0,0,0,0,0,0,0,0,0
8,id_95,3.289474e-08,0.093053,0.0,7.500000e-07,8.223684e-08,9.868421e-08,1.274520e-08,pu,ce,...,0,0,0,0,0,0,0,0,0,0
9,id_97,5.500000e-07,0.158401,0.0,2.105000e-06,2.769737e-07,5.500000e-07,3.138505e-07,da,tn,...,0,0,0,0,0,0,0,0,0,0


In [65]:
# preprocessing 
def preprocessing(df):
    # remove str cat variables..
    for i in range(18):
        st = 'cat_var_' + str(i+1)
        df = df.drop([st], axis = 1)
    
    # mean sd normalization for sold col...
#     for i in range(7):
#         st = 'num_var_' + str(i+1)
#         mean = np.mean(df[st])
#         sd = np.sqrt(np.var(df[st]))
#         df[st] = (df[st] - mean) / sd
    
    df = df.drop(['transaction_id'], axis=1)        
    return df

In [66]:
processed_data = preprocessing(dataset_train_validate)
processed_data

,num_var_1,num_var_2,num_var_3,num_var_4,num_var_5,num_var_6,num_var_7,cat_var_19,cat_var_20,cat_var_21,...,cat_var_34,cat_var_35,cat_var_36,cat_var_37,cat_var_38,cat_var_39,cat_var_40,cat_var_41,cat_var_42,target
0,2.302632e-08,0.040182,0.0,1.800000e-07,2.302632e-08,2.368421e-08,1.115205e-08,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,7.965789e-06,0.157872,0.0,2.105000e-06,2.769737e-07,7.965789e-06,2.433058e-06,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,7.828947e-08,0.089140,0.0,3.550000e-07,4.671053e-08,1.052632e-07,4.276014e-07,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,7.894737e-08,0.227239,0.0,1.050000e-06,1.381579e-07,2.190789e-07,1.848054e-08,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,3.321053e-06,0.160410,0.0,2.105000e-06,2.769737e-07,3.340789e-06,2.152983e-06,1,0,1,...,0,0,0,0,0,0,0,0,0,0
5,1.953947e-07,0.078566,0.0,3.550000e-07,4.671053e-08,4.407895e-08,9.463310e-08,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,1.736842e-07,0.600508,0.0,4.575000e-06,5.986842e-07,1.736842e-07,8.411831e-08,1,0,1,...,0,0,0,0,0,0,0,0,0,0
7,5.789474e-08,0.152374,0.0,2.105000e-06,2.769737e-07,5.789474e-08,2.453451e-08,1,0,1,...,0,0,0,0,0,0,0,0,0,0
8,3.289474e-08,0.093053,0.0,7.500000e-07,8.223684e-08,9.868421e-08,1.274520e-08,0,1,1,...,0,0,0,0,0,0,0,0,0,0
9,5.500000e-07,0.158401,0.0,2.105000e-06,2.769737e-07,5.500000e-07,3.138505e-07,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [67]:
train_set = processed_data[:300000]
test_set = processed_data[300000:]

In [68]:
# visualization
plt.plot(processed_data['num_var_6'])
plt.show()

KeyboardInterrupt: 

In [ ]:
y_train = train_set['target'].values
x_train = train_set.drop(['target'], axis=1)

y_valid = test_set['target'].values
x_valid = test_set.drop(['target'], axis=1)

print x_train.shape, y_train.shape,x_valid.shape, y_valid.shape

In [ ]:
d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)

# free memory
del x_train, x_valid; gc.collect()

In [ ]:
print('Training ...')

params = {}
params['eta'] = 0.02
params['objective'] = 'reg:linear'
params['eval_metric'] = 'mae'
params['max_depth'] = 16
params['silent'] = 1

watchlist = [(d_train, 'train'), (d_valid, 'valid')]
clf = xgb.train(params, d_train, 100000, watchlist, early_stopping_rounds=100, verbose_eval=10)

# del d_train, d_valid

In [57]:
print('Building test set ...')

submission = {}

X = dataset_test['transaction_id']
x_test = preprocessing(dataset_test)

d_test = xgb.DMatrix(x_test)
p_test = clf.predict(d_test)

submission['target'] = p_test
# submission['transaction_id'] = X
print submission

Building test set ...
{'target': array([ 0.06248462,  0.1133295 ,  0.15091196, ...,  0.15471166,
        0.0796892 ,  0.14024365], dtype=float32)}


In [62]:
# df construction
sub_df = pd.DataFrame(data={'transaction_id':X}, index=np.arange(len(submission['target'])))
sub_df['target'] = p_test

In [64]:
print('Writing csv ...')
sub_df.to_csv('submission.csv', index=False, float_format='%.4f')

Writing csv ...


In [63]:
sub_df

,transaction_id,target
0,id_1,0.062485
1,id_6,0.113329
2,id_9,0.150912
3,id_14,0.271469
4,id_15,0.137367
5,id_19,0.107901
6,id_20,0.124408
7,id_24,0.082330
8,id_25,0.070677
9,id_28,0.162615
